### Learning Objectives

- Matplotlib basics syntax;
- Customizing a figure;
- Understanding the `figure`, `axes` and how to create `subplots`;
- Plotting using matplotlib wrappers (pandas, xarray, etc).

![](https://holoviz.org/assets/landscape_hv_nx.png)

### Basic matplotlib

![](data/mpl_anatomy.png)

In [ ]:
# Try qt5 or notebook later.
# You'll have to restart the kernel first!
%matplotlib inline

import matplotlib.pyplot as plt

In [ ]:
import matplotlib


matplotlib.get_backend()

In [ ]:
fig = plt.figure()

In [ ]:
ax = plt.axes()

In [ ]:
fig, ax = plt.subplots()

In [ ]:
fig, ax = plt.subplots(
    figsize=(8, 4),
    ncols=2,
)

In [ ]:
import pandas as pd


url = ("https://raw.githubusercontent.com/ocefpaf/"
       "2018-Jun-SWC-Floripa/master/data")

fname = "dados_pirata.csv"
df = pd.read_csv(
    f"{url}/{fname}",
    index_col='datahora',
    parse_dates=True,
    na_values=-99999,
)

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.columns = ['{0:0>3}'.format(col.split('_')[1]) for
              col in df.columns]

df.sort_index(axis=1, inplace=True)
df.head(10)

In [ ]:
fig, ax = plt.subplots()

z = df.columns.astype(float)

ax.plot(df.iloc[0, :], -z, "-o");

## What are we plotting? Let's label!

In [ ]:
fig, ax = plt.subplots()

ax.plot(df.iloc[0, :], -z, "-o")

fig.suptitle("PIRATAS")
ax.set_title("Mooring #42")
ax.set_xlabel(r"$\theta$ (℃)")
ax.set_ylabel("Depth (m)");

### We can customize the `ticks`...

In [ ]:
fig, ax = plt.subplots()

ax.plot(df.iloc[0, :], -z, "-o")
ax.set_yticks([0, -100, -400, -600]);

###  ... and the `yticklabels()`.

In [ ]:
fig, ax = plt.subplots()

ax.plot(df.iloc[0, :], z, "-o")
ax.invert_yaxis()
ax.set_yticks([0, 100, 500]);
ax.set_yticklabels(["surface", "mixed layer", "thermocline"]);

Exercise: We used `df.iloc[0, :]` to  plot the first time index versus the all depths.
Create for a time series (all time indexes) for a single depth and customize it.

In [ ]:
%load 00-mpl-basics-00-sol.py

### Plotting directly from pandas

In [ ]:
ax = df.plot(figsize=(11, 2.75))
ax.legend(loc=(1.05, 0))
ax.grid(True);

### This allow for a compact processing/plotting code

In [ ]:
plot = {"marker": ".", "linestyle": "none", "figsize": (11, 3)}
interp = {"method": "time", "limit": 10}

df["001"].plot(**plot);
df["001"].interpolate(**interp).plot(alpha=0.65);

Exercise: check the `plt.subplots` documentation and create a subplot with two rows and 1 column. Plot different depths in the rows.

In [ ]:
%load 00-mpl-basics-01-sol.py

Exercise: load the file `15t30717.3f1` from the `url` below and plot the `u` and `v` time-series.

url = "https://raw.githubusercontent.com/ocefpaf/2018-Jun-SWC-Floripa/master/data/15t30717.3f1"

Note that this dataset is not labeled! (Welcome to the messy world of bad/absent metadata.)

Use `names = ["j", "u", "v", "temp", "sal", "y", "mn", "d", "h", "mi"]` in pandas to identify the columns.

bonus points: filter the tidal signal using pandas `rolling` method.

In [ ]:
%load 00-mpl-basics-02-sol.py

In [ ]:
%load 00-mpl-basics-03-sol.py

### Plotting slices and surfaces

For more examples see <https://ioos.github.io/notebooks_demos/notebooks/2019-03-08-grids-temperature/>.

In [ ]:
import xarray as xr


url = ("http://tds.marine.rutgers.edu/thredds/dodsC/"
       "roms/doppio/2017_da/avg/Averages_Best")
ds = xr.open_dataset(url)

In [ ]:
temp = ds.filter_by_attrs(
    standard_name="sea_water_potential_temperature"
)
temp.info()

In [ ]:
surface = temp.isel(time=-1, s_rho=-1).to_array()

surface.plot(
    x="lon_rho",
    y="lat_rho",
);

In [ ]:
import cartopy.crs as ccrs
from palettable.cmocean.sequential import Thermal_20

cmap = Thermal_20.mpl_colormap

ax = plt.axes(projection=ccrs.PlateCarree())
surface.plot(ax=ax,
             x="lon_rho",
             y="lat_rho",
             cmap=cmap)

ax.coastlines(resolution="10m");

Exercise: change the `isel` call we did above,

```python
temp.isel(time=-1, s_rho=-1).to_array()
```

and create a time-series plot at a certain point and another slice at any given depth. Was it easy to figure out the point and the depth?

Challenge: perform a temperature slice at a given depth (250 m).

In [ ]:
%load 00-mpl-basics-04-sol.py

In [ ]:
%load 00-mpl-basics-05-sol.py

In [ ]:
%load 00-mpl-basics-06-sol.py

Exercise: choose a model from  https://eds.ioos.us, load with `xarray`, and plot a surface layer slice of any property.

### Vertical sections

In [ ]:
url = ("http://eds.ioos.us/eds_thredds/dodsC/"
       "EDS/HYCOM_3D_R1/hycom_3d_r1_2019082600.nc")

ds = xr.open_dataset(url, decode_times=False)
section = ds["salinity"].isel(time=0)
section = section.sel(lon=302, method='nearest')

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))
cs = section.plot(ax=ax)
ax.invert_yaxis()

If you need more complex sections, like comparing the path of a glider with a model check [this notebook](https://nbviewer.jupyter.org/gist/ocefpaf/b952cb560f4a508b964825b1b53c78da). You'll notice that it does not use `xarray`, the main reason is the lack of support for vertical coordinates. [More on that on here](https://gist.github.com/ocefpaf/18e8fe29005c148fe0e3).

### Want to know how to perform vertical slices on numerical models?

### This is only be beginning!

There are many tutorials on matplotlib out there. Here are a few resouces:

- https://github.com/matplotlib/GettingStarted
- https://matplotlib.org/gallery.html
- https://github.com/rougier/matplotlib-cheatsheet

In [ ]:
import hvplot.pandas


df[["v", "high", "low"]].hvplot()